# Nuoseklių sakinių klasifikavimas atsitiktinių imčių medicininių tyrimų santraukose

## Santrauka

### Tikslas

Sukurti natūralios kalbos apdorojimo (*angl. Natural Language Processing*, **NLP**) modelį, kuris palengvintų medicininių tyrimų publikacijų santraukų skaitymą.

### Metodas

### Rezultatas

### Išvada

### Bibliotekų importavimas

In [11]:
import os
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

2023-03-13 16:01:20.584540: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 1. Įvadas

2010 metais straipsnyje autorius *A. Jinha* teigė, kad tuo metu buvo publikuota daugiau nei **50 milijonų mokslinių straipsnių**, iš kurių maždaug pusė biomedicinos temomis. 2014 metais atliktame tyrime *M. Khabsa ir C.L. Giles* suskaičiavo: akademinėse paieškose *Google Scholar* ir *Microsoft Academic Search* yra prieinama apie **114 milijonų anglų kalba parašytų mokslinių publikacijų**.

Kiekvienais metais publikuojamų atsitiktinių imčių medicininių tyrimų skaičius tik auga, todėl medicinos specialistams tampa vis sunkiau aptikti jiems reikiamą informaciją. Pavyzdžiui, kai tyrinėtojai ieško ankstesnės literatūros, jie dažnai perskaito santraukas, kad greitai patikrintų, ar straipsniai atitinka jų kriterijus. Šis procesas yra lengvesnis jei santrauka yra **struktūrizuota**, t.y. padalinima į semantines antraštes (etikėtes), tokias kaip **tikslas, metodas, rezultatas ir išvada**. 

Kiekvieną santraukos sakinį suskirstę į atitinkamą antraštę galime žymiai sutrumpinti norimos informacijos radimo laiką, šis procesas vadinamas nuoseklių sakinių klasifikavimu. Šiame darbe įgyvendinama dirbtiniais neuroniniais tinklais pagrįsta sistema, skirta nuoseklaus sakinio klasifikavimo užduočiai panaudojant [PubMed 200k RCT](https://arxiv.org/pdf/1710.06071.pdf) duomenų rinkinį. Darbas atliekamas remiantis autorių Franck Dernoncourt, Ji Young Lee ir Peter Szolovits 2017 metų publikacija "[Neural Networks for Joint Sentence Classification in Medical Paper Abstracts](https://arxiv.org/pdf/1612.05251)".

TENSORFLOW

## 2. Duomenų rinkinys (*angl. dataset*)


[PubMed 200k RCT](https://arxiv.org/pdf/1710.06071.pdf) duomenų rinkinys sudaryta iš viešai prieinamos bibliotekos [PubMed](https://pubmed.ncbi.nlm.nih.gov/) įrašų ir yra skirtas nuosekliam sakinių klasifikavimui atsitiktinės imties kontroliuojamų medicininių tyrimų santraukose. Duomenų rinkinys susideda iš maždaug 200 000 santraukų, kurias sudaro apie  2,3 mln. sakinių. Kiekvienas sakinys yra pažymėtas etikėte, nurodančia sakinio reikšmę santraukoje:
- **background** - kilmė, ištakos
- **objective** - tikslas
- **methods** - metodas
- **results** - rezultatas
- **conclusions** - išvada

Duomenų rinkinio publikavimas turi dvi paskirtis:
- dauguma esamų nuoseklių sakinių klasifikavimui skirtų duomenų rinkinių yra nedideli arba viešai neprieinami, todėl sudėtingiau sukurti tikslesnius klasifikavimo algoritmus;
- tyrėjams reikia geresnių įrankių efektyvesniam mokslinės literatūros skaitymui.


### 2.1. Duomenų rinkinio atsiuntimas

Duomenų rinkinys yra viešai prienamas vieno iš autorių [GitHub repozitorijoje](https://github.com/Franck-Dernoncourt/pubmed-rct)

In [ ]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct

In [5]:
!ls pubmed-rct

PubMed_200k_RCT
PubMed_200k_RCT_numbers_replaced_with_at_sign
PubMed_20k_RCT
PubMed_20k_RCT_numbers_replaced_with_at_sign
README.md


Pagrindinis duomenų rinkinys **PubMed_200k_RCT** kai kuriais atvejais gali būti per didelis, todėl papildomai yra pateikiamas **PubMed 20k RCT** rinkinys, sudarytas iš 10% pagrindiniame rinkinyje esančių vėliausiai publikuotų tyrimų santraukų.

Taip pat pateikiamos dvi kiekvieno rinkinio versijos:
- visas originalus tekstas,
- tekste esantys skaičiai pakeisti @ simboliu.


### 2.2. Duomenų rinkinio formatas

In [3]:
!ls pubmed-rct/PubMed_20k_RCT/

dev.txt  test.txt  train.txt


In [4]:
!ls pubmed-rct/PubMed_200k_RCT/

dev.txt  test.txt  train.7z


Rinkinio aplankuose mes rasime po tris failus:
- **train.txt** arba **train.zip** - modelio mokymo santraukų failas
- **test.txt** - modelio testavimo santraukų rinkinys
- **dev.txt** - modelio įvertinimo (*angl. validation*) santraukų rinkinys

### 2.3. Duomenų rinkinio failo formatas

Kiekvienas failas turi tą patį formatą:
- PMID (PubMed ID) eilutė - santraukos pradžia, publikacijos ID PubMed bibliotekoje;
- santraukos sakiniai atskirose eilutėse, kiekvienos pradžioje didžiosiomis raidėmis sakinio etiketė ir santraukos sakinys atskirti tabuliacijos simboliu (*angl. tab*);
- tuščia eilutė - santraukos pabaiga.


In [12]:
FOLDER_PATH = "pubmed-rct/PubMed_20k_RCT/"
FILE_PATH = os.path.join(FOLDER_PATH, "test.txt")

# nuskaitom visas failo eilutes į sąrašą file_lines
with open(FILE_PATH, "r") as f:
    file_lines = f.readlines()

# išvedam 20 pirmų failo eilučių
file_lines[:20]

['###24845963\n',
 'BACKGROUND\tThis study analyzed liver function abnormalities in heart failure patients admitted with severe acute decompensated heart failure ( ADHF ) .\n',
 'RESULTS\tA post hoc analysis was conducted with the use of data from the Evaluation Study of Congestive Heart Failure and Pulmonary Artery Catheterization Effectiveness ( ESCAPE ) .\n',
 'RESULTS\tLiver function tests ( LFTs ) were measured at 7 time points from baseline , at discharge , and up to 6 months follow-up .\n',
 'RESULTS\tSurvival analyses were used to assess the association between admission Model of End-Stage Liver Disease Excluding International Normalized Ratio ( MELD-XI ) scores and patient outcome.There was a high prevalence of abnormal baseline ( admission ) LFTs ( albumin 23.8 % , aspartate transaminase 23.5 % , alanine transaminase 23.8 % , and total bilirubin 36.1 % ) .\n',
 "RESULTS\tThe percentage of patients with abnormal LFTs decreased significantly from baseline to 6-months ' follow-u

### Dataset padalinimas

Pagrindinį PubMed 200k RCT duomenų rinkinį sudaro 195 654 santraukos, kurios atsitiktine tvarka suskirstytos į tris rinkinius:
validation (modelio įvertinimo) - 2500 santraukų,
test (testavimo/modelio bandymo) - 2500 santraukų,
train (modelio mokymo) - likusios 190 654 santraukos.

Pagrindinis duomenų rinkinys kai kuriais atvejais gali būti per didelis, todėl papildomai yra pateikiamas PubMed 20k RCT rinkinys, kuriame 15 tūkst. train ir po 2500 test bei validation santraukų. Šis rinkinys buvo sudarytas iš pagrindiniame rinkinyje esančių vėliausiai publikuotų tyrimų santraukų.

Taip pat pateikiamos dvi kiekvieno rinkinio versijos:
visas originalus tekstas,
tekste esantys skaičiai pakeisti @ simboliu.


## Bazinis modelis